In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext sql 
%pylab inline
%matplotlib inline

import matplotlib.text as text

import scipy.interpolate as interp
import pandas as pd
#from mp.txpepa import *
from sklearn import (manifold, datasets, decomposition, ensemble, lda,
                     random_projection)
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances
import statsmodels.api as sm
import numpy.linalg as LA
import bio.hts.traj as traj
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
stats = importr('stats')
from sklearn.neighbors import KNeighborsClassifier

DAT_DIR = '/share/home/ishah/projects/Chem/data/tables/'
PKL_DIR = '/share/home/ishah/projects/Chem/data/pickle/'
RES_DIR = '/share/home/ishah/projects/Chem/data/results/'
FIG_DIR = '/share/home/ishah/projects/Chem/figs/readacross/'

import pickle

tmstmp = time.strftime("%m-%d-%Y",time.localtime())
tmstmp

In [ ]:
from bio.data.genra import *

DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='genra_v3')
DB.collection_names()

In [ ]:
def calcPerf2(n_pos,n_neg,sens,spec,sn_sd=None,sp_sd=None):
    n = n_pos+n_neg
    TP = int(sens * n_pos)
    TN = int(spec * n_neg)
    FN = n_pos - TP
    FP = n_neg - TN
    
    if (TP+FP)==0 or (TN+FN)==0: return 
    
    PPV= 1.0*TP/(TP+FP)
    NPV= 1.0*TN/(TN+FN)
    FDR= 1 - PPV
    
    se=sp_se=sn_se=None
    
    if sn_sd and sp_sd:
        sn_se=sn_sd/np.sqrt(n)
        sp_se=sn_sd/np.sqrt(n)
        se = np.sqrt(sn_se**2 + sp_se**2)*np.sqrt(n)
        
    return dict(#n_pos=n_pos,n_neg=n_neg,
                TP=TP,TN=TN,FP=FP,FN=FN,
                #Sens=sens,Spec=spec,
                PPV = PPV, NPV=NPV, FDR=FDR,
                se=se
               )


In [ ]:
q = {'auc':{'$gte':0.6},'n_pos':{'$gte':2},'n_neg':{'$gte':2}}
print DB.genra_perf_v1.find(q).count()
DB.genra_perf_v1.find_one(q)


In [ ]:
P0 = DB.genra_perf_v1.find_one(q)
calcPerf2(P0['n_pos'],P0['n_neg'],P0['sn_max'],P0['sp_max']
         )

In [ ]:
from util.jupyter import *

In [ ]:
DB.genra_perf_v2.drop()
q = {'auc':{'$gte':0.2},'n_pos':{'$gte':2},'n_neg':{'$gte':2}}
N = DB.genra_perf_v1.find(q,dict(_id=0)).count()

for P0 in log_progress(DB.genra_perf_v1.find(q,dict(_id=0)),every=10,size=N):
    P1=calcPerf2(P0['n_pos'],P0['n_neg'],P0['sn_max'],P0['sp_max'])
    if not P1: continue
    P0.update(P1)
    DB.genra_perf_v2.insert_one(P0)

In [ ]:
DB.genra_perf_v2.count()

In [ ]:
DB.genra_perf_v2.find_one()

In [ ]:
len(DB.genra_perf_v2.distinct('cl'))

In [ ]:
r=dict(_id=0,se=0)
Perf0 = (pd.DataFrame(list(DB.genra_perf_v2.find({},r)))
         .rename(columns=dict(sim_type='dt',sn_max='SN_max',sp_max='SP_max',auc='AUC',PPV='PPV_max',NPV='NPV_max'))
        )

In [ ]:
def Unq_Cl(x): return len(np.unique(x))

In [ ]:
P1 = (Perf0.query("auc_pval<0.1")
      .groupby(['effect','dt'])
      .aggregate(dict(PPV=[np.median,np.std],
                      NPV=[np.median,np.std],
                      SN_max=[np.median,np.std],
                      SP_max=[np.median,np.std],
                      AUC=[np.median,np.std],
                      cl= Unq_Cl ))
     )
P1.head(30)

In [ ]:
P1 = (Perf0#.query("auc_pval<0.1")
      .groupby(['effect','dt'])
      .aggregate(dict(PPV_max=dict(mn=np.mean,sd=np.std),
                      NPV_max=dict(mn=np.mean,sd=np.std),
                      SN_max=dict(mn=np.mean,sd=np.std),
                      SP_max=dict(mn=np.mean,sd=np.std),
                      AUC=dict(mn=np.mean,sd=np.std),
                      cl= Unq_Cl))
     )
P1.head(30)

In [ ]:
Res = []
for effect,Pi in P1.groupby(level=0):
    Res.append(Pi.sort_values(('AUC','mn')).tail(1))
    print effect,dt,Pi.shape

In [ ]:
P2 = pd.concat(Res)

In [ ]:
P2 = (P2[['cl','AUC','SN_max','PPV_max','SP_max']]
      .round(decimals=2)
      .rename(dict(SN_max='sens',SP_max='spec',PPV_max='ppv'))
     )

In [ ]:
P2.head()

In [ ]:
C = [i+'_'+j for (i,j) in list(P2.columns)]
P2.columns=C


In [ ]:
P2.head()

In [ ]:
P2['res2_sn']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.SN_max_mn,x.SN_max_sd),axis=1)
P2['res2_sp']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.SP_max_mn,x.SP_max_sd),axis=1)
P2['res2_ppv']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.PPV_max_mn,x.PPV_max_sd),axis=1)
P2['res2_auc']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.AUC_mn,x.AUC_sd),axis=1)

In [ ]:
P2.head()

In [ ]:
X = P2.sort_values('AUC_mn',ascending=False)[['cl_Unq_Cl','res2_auc','res2_sn','res2_sp','res2_ppv']]

#X.columns = "Classifier ID	Study:Target	dt	n	ML Alg	F1	Sensitivity	Specificity	PPV".split("\t")

X.to_csv('/share/home/ishah/tmp/genra-perf-with-ppv-summary-v2.csv')

#ew = pd.ExcelWriter('/share/home/ishah/tmp/genra-perf-with-ppv-summary-v2.xlsx', )
#X.to_excel(ew)
#ew.save()
#for x in [i for i in X if i.startswith('res2')]:
#    X[x] = X[x].str.decode('ascii').str.encode('utf-8')
    

